In [1]:
import pandas as pd

# Kano Analysis
## Kano analysis is a framework for prioritizing product features from a backlog
## To conduct a Kano analysis, we need:
- A set of users to survey
- A small set of features to study
- Two questions about each feature:
    - Functional question: How do you feel when Feature X is available?
    - Dysfunctional question: How do you feel when Feature X is unavailable?
- A limited list of valid answers to each question

In [2]:
# restate the valid answers for each question
potentialAnswers = ['Like it', 'Expect it', 'Don\'t care', 'Can live with it', 'Dislike it']

# read in (simulated) survey data
surveyData = pd.read_csv('./data/Survey Data.csv')

# convert answers to ordered categorical features for ease of evaluation
surveyData['Functional Answer'] = pd.Categorical(surveyData['Functional Answer']
                                                 , categories = potentialAnswers, ordered = True)
surveyData['Dysfunctional Answer'] = pd.Categorical(surveyData['Dysfunctional Answer']
                                                    , categories = potentialAnswers, ordered = True)

surveyData

,User,Feature,Functional Answer,Dysfunctional Answer
0,User 1,Feature 1,Don't care,Don't care
1,User 2,Feature 1,Expect it,Dislike it
2,User 3,Feature 1,Like it,Can live with it
3,User 4,Feature 1,Like it,Can live with it
4,User 5,Feature 1,Can live with it,Expect it
...,...,...,...,...
495,User 96,Feature 5,Expect it,Can live with it
496,User 97,Feature 5,Like it,Dislike it
497,User 98,Feature 5,Don't care,Don't care
498,User 99,Feature 5,Expect it,Can live with it


## Kano analysis is a framework for prioritizing product features from a backlog
- Product users are surveyed about their feelings towards potential features
- Each potential feature is presented in both a functional and dysfunctional scenario
- Features are categorized based on how much each user benefits from the presence of the feature AND suffers from the absence of it:
    - Features that users didn't necessarily expect, but are delighted and impresesd by, are considered "attractive"
    - Features that users like having and dislike not having are called "performance" features. Over time, novel features once considered "attractive" might move into this category
    - The features that users only notice when they're missing are called "must-haves"
    - Features that users are neutral about the presence or absence of are called "indifferent"
    - Occasionally, a user might give contradictory answers about a feature, saying they like having and not having the feature equally. These results are considered "questionable" and generally excluded from analysis
    - In the unexpected event that users give more favorable responses about the absence of a feature than the presence of it, that's considered a "reverse" feature - and it's probably a good idea to revisit its design!

In [3]:
kanoRules = pd.DataFrame(
    [['Questionable', 'Attractive', 'Attractive', 'Attractive', 'Performance']
    , ['Reverse', 'Questionable', 'Indifferent', 'Indifferent', 'Must-have']
    , ['Reverse', 'Indifferent', 'Indifferent', 'Indifferent', 'Must-have']
    , ['Reverse', 'Indifferent', 'Indifferent', 'Questionable', 'Must-have']
    , ['Reverse', 'Reverse', 'Reverse', 'Reverse', 'Questionable']
    ]
    , columns = potentialAnswers
    , index = potentialAnswers
)
kanoRules

,Like it,Expect it,Don't care,Can live with it,Dislike it
Like it,Questionable,Attractive,Attractive,Attractive,Performance
Expect it,Reverse,Questionable,Indifferent,Indifferent,Must-have
Don't care,Reverse,Indifferent,Indifferent,Indifferent,Must-have
Can live with it,Reverse,Indifferent,Indifferent,Questionable,Must-have
Dislike it,Reverse,Reverse,Reverse,Reverse,Questionable


In [4]:
# map each survey result to a Kano category
surveyData['Kano Category'] = surveyData.apply(lambda x: kanoRules.loc[x['Functional Answer'], x['Dysfunctional Answer']], axis = 1)

# convert answers to ordered categorical features for ease of evaluation
kanoCategories = ['Questionable', 'Reverse', 'Indifferent', 'Attractive', 'Performance', 'Must-have']
surveyData['Kano Category'] = pd.Categorical(surveyData['Kano Category']
                                             , categories = kanoCategories
                                             , ordered = True)

surveyData

,User,Feature,Functional Answer,Dysfunctional Answer,Kano Category
0,User 1,Feature 1,Don't care,Don't care,Indifferent
1,User 2,Feature 1,Expect it,Dislike it,Must-have
2,User 3,Feature 1,Like it,Can live with it,Attractive
3,User 4,Feature 1,Like it,Can live with it,Attractive
4,User 5,Feature 1,Can live with it,Expect it,Indifferent
...,...,...,...,...,...
495,User 96,Feature 5,Expect it,Can live with it,Indifferent
496,User 97,Feature 5,Like it,Dislike it,Performance
497,User 98,Feature 5,Don't care,Don't care,Indifferent
498,User 99,Feature 5,Expect it,Can live with it,Indifferent


In [5]:
# count survey results by feature
featureData = pd.pivot_table(surveyData, index = 'Feature', columns = 'Kano Category', values = 'User', aggfunc = 'count')

# identify the mode Kano category for each feature
featureData['Primary Kano Category'] = featureData.apply(lambda x: kanoCategories[x.argmax()], axis = 1)

featureData

Kano Category,Questionable,Reverse,Indifferent,Attractive,Performance,Must-have,Primary Kano Category
Feature,,,,,,,
Feature 1,2,9,31,27,8,23,Indifferent
Feature 2,1,17,32,21,6,23,Indifferent
Feature 3,3,15,23,24,2,33,Must-have
Feature 4,3,9,36,31,4,17,Indifferent
Feature 5,6,11,20,28,7,28,Attractive
